In [29]:
# !pip install elasticsearch
# !pip install tiktoken

In [30]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

In [2]:
!curl localhost:9200

{
  "name" : "44705cc2e00c",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "QKOnGrGpSIa__8vctvaiIw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
url='http://localhost:9200'
es_client = Elasticsearch(url)
es_client.info()

ObjectApiResponse({'name': '44705cc2e00c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QKOnGrGpSIa__8vctvaiIw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
# !curl -XGET 'localhost:9200/course-questions?pretty'

In [5]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course_questions'
es_client.indices.create(index=index_name , body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index= index_name, document=doc)

100%|█████████████████████| 948/948 [00:22<00:00, 43.04it/s]


In [10]:
query= "How do I execute a command in a running docker container?"

In [11]:
def elastic_search(query):
    search_query ={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response=es_client.search(index=index_name,body=search_query)
    result_docs=[]
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
   
    context=""
    for doc in search_results:
        context = context + f"""
Q: {doc["question"]}
A: {doc["text"]}
""".strip()

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
context=elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'm

In [24]:
prompt=build_prompt(query, context)

In [25]:
print(len(prompt))

1482


In [32]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

327

In [8]:
# Q1:What's the version.build_hash value?
# A1: "42f05b9372a9a4a470db3b52817899b99a76ee73"

# Q2:Which function do you use for adding your data to elastic?
# A2: index

# Q3:What's the score for the top ranking result?
# A3: 74.05

# Q4:What's the 3rd question returned by the search engine?
# A4:How do I copy files from a different folder into docker container’s working directory?

# Q5:What's the length of the resulting prompt?
# A5: 1462

# Q6:Use the encode function. How many tokens does our prompt have?
# A6: 322